In [1]:
!pip install duckdb==1.4.1 --upgrade

# run sql show tables using duckdb

In [2]:
import duckdb
import time

In [3]:
CATALOG_URL = "http://host.docker.internal:8282/catalog"
KEYCLOAK_TOKEN_URL = "http://host.docker.internal:30080/realms/iceberg/protocol/openid-connect/token"

In [4]:
duckdb.sql("INSTALL ICEBERG;");
duckdb.sql("UPDATE EXTENSIONS;");
duckdb.sql("LOAD ICEBERG;");

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
duckdb.sql(f"""CREATE SECRET lakekeeper_secret (
    TYPE ICEBERG,
    CLIENT_ID 'starrocks',
    CLIENT_SECRET 'X5IWbfDJBTcU1F3PGZWgxDJwLyuFQmSf',
    OAUTH2_SCOPE 'lakekeeper',
    OAUTH2_SERVER_URI '{KEYCLOAK_TOKEN_URL}'
)
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [6]:
duckdb.sql(f"""attach 'demo' as my_datalake (
	type ICEBERG,
	ENDPOINT '{CATALOG_URL}',
	SECRET lakekeeper_secret
)
""");

# select is fast

In [7]:
start_time = time.time()
duckdb.sql("select * from my_datalake.namespace10.my_table2;").show()
print(f"cost: {time.time() - start_time:.4f} s")

┌───────┬─────────┐
│  id   │  data   │
│ int64 │ varchar │
├───────┴─────────┤
│     0 rows      │
└─────────────────┘

cost: 0.2789 s


# SHOW ALL TABLES is slow

In [8]:
start_time = time.time()
duckdb.sql("show tables from my_datalake.namespace10;").show()
print(f"cost: {time.time() - start_time:.4f} s")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ my_table1  │
│ my_table10 │
│ my_table11 │
│ my_table12 │
│ my_table13 │
│ my_table14 │
│ my_table15 │
│ my_table16 │
│ my_table17 │
│ my_table18 │
│     ·      │
│     ·      │
│     ·      │
│ my_table28 │
│ my_table29 │
│ my_table3  │
│ my_table30 │
│ my_table4  │
│ my_table5  │
│ my_table6  │
│ my_table7  │
│ my_table8  │
│ my_table9  │
├────────────┤
│  30 rows   │
│ (20 shown) │
└────────────┘

cost: 88.1427 s
